In [99]:
import pandas as pd
import numpy as np
import annoy
import nltk
import re
from gensim.models import word2vec
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [100]:
data = pd.read_csv('data', sep='@')
vect_len = 500

In [101]:
# train on 2015 and 2016, test on 2018
# ex = pd.concat((data[data.Year == 2016], data[data.Year == 2015], data[data.Year == 2014]), axis=0, ignore_index=True)
ex = data.copy()
# test = data[data.Year == 2018]
ex = ex.dropna()

In [102]:
stops = set(stopwords.words("english"))
lem = PorterStemmer()
def convert(abstract):
    '''Преобразует текст для w2v'''
    text = re.sub("[^a-z.!?]"," ", abstract)
    words = text.lower().split()
    words = [w for w in words if not w in stops and len(w) >= 1]
#     words = [lem.stem(w) for w in words ]
    return ' '.join(words)

In [103]:
ex['cl_abst'] = ex.Abstract.apply(convert)


In [104]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
def tok_abstr(abstr, tokenizer):
    return [ s.replace('.', '') for s in tokenizer.tokenize(abstr.strip())]
sentences  = [sent.replace('.', '').split() for row in ex.cl_abst for sent in tok_abstr(row, tokenizer) if len(sent) >= 1]
model = word2vec.Word2Vec(sentences, size=vect_len, window=20, workers=4)

ex['new'] = ex.cl_abst.apply(lambda x : tok_abstr(x, tokenizer))
words = set(model.wv.index2word)

In [105]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/home/alex/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [ ]:
def avg_single_sentence(snt, words, model, size, vect, tfidf):
    vector = np.zeros(size)
    counter = 0
    for w in snt:
        if w in words:
            vector += model[w]
            counter += 1
    return vector / counter

In [ ]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tfidf = vect.fit_transform(ex.cl_abst)
true_weigth = lambda x : max(tfidf[:, x]).todense() * np.log10(tfidf.shape[0] / tfidf[:, x].count_nonzero())
weight = {}
idx = 0
for w in vect.get_feature_names():
    weight[w] = true_weigth(idx)


In [43]:
# ex['vectors'] = ex.new.apply(lambda x : avg_single_sentence(x, words, model, vect_len))
# ex['vectors'] = pd.DataFrame(data_mean,columns=['vectors'])
test = ['dark', 'matter', 'gravity', 'field',  'repulsion']

vec = tfidf_vectorizer(w2v).fit(test).transform(test)

vect_len = vect_len
t = annoy.AnnoyIndex(vect_len)
for i, v in enumerate(data_mean):
    t.add_item(i, v)
t.build(20)


True

In [44]:
t.get_nns_by_vector(vec, 10, include_distances=True)

IndexError: Vector has wrong length (expected 500, got 5)

In [123]:
ex.Link[2139]

'https://elibrary.ru/item.asp?id=24043443'

In [46]:
vec.shape

(5, 500)